In [4]:
% reload_ext autoreload
% autoreload 2
% matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import utils
from lib.pipeline import resized

C = utils.get_config('./myconfig.ini')

# Artstyle

The collection of WikiArt paintings cropped to `224x224` at the centre. See `MNIST.ipynb` for introduction.

## Resized

### Loading dataset

In [2]:
traindata = resized.Data(root=C.paths.root, info_csv=C.paths.train_csv, train=True)
print('{} Instances. {} Classes'.format(len(traindata), len(traindata.encoder.classes_)))

79433 Instances. 136 Classes


### Training - AlexNet

In [ ]:
%%time
EPOCHS = 1
XVAL = 0.05
BATCH_SIZE = 128
LRATE = 1e-3
MOMENTUM = 0.9
DEVICE = 'cuda:0'

net = resized.Net(pretrained=True).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=LRATE, momentum=MOMENTUM)

model = resized.Model(net=net, criterion=criterion, optimizer=optimizer, device=DEVICE)
model.train(dataset=traindata, batch_size=BATCH_SIZE, epochs=EPOCHS, xvalidate=XVAL, num_workers=2)

# model.save('resized-alexnet.pt')